## Malaria transmission process parameters

In [1]:
params = {
    "rainfall_multiplier" : 1.0, # rainfall to mosquito linear multiplication factor (per day)
    "rainfall_offset" : 0.0, # rainfall to mosquito offset
    "c" : 1.0, # inoculation effiency for mosquitoes
    "b": 0.5, # inoculation efficiency for humans
    "hbi" : 0.5, # human blood index
    "r" : 0.0284, # human recovery rate (per day)
    "hia" : 15.0, # number of days after infection when humans become infectious
    "Tg" : 9.0, #  Gonotrophic threshold temperature 
    "Dg" : 37.0, # Gonotrophic cycle length in degree days
    "Ts" : 18.0, # Sporogonic threshold temperature 
    "Ds" : 111.0, # Sporoogonic cycle length in degree days
    "stype" : 0 # Survival type (valid values are 0 = Martens, 2 = Craig)
}

In [2]:
import pandas as pd
from models.R0_model import *


/Users/juliankuehnert/.pyenv/versions/3.8.6/envs/ungana/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
dataset = 'seasonal_forecasts.csv'

In [5]:
df = pd.read_csv('datasets/'+dataset)
df.head()

,forecast,precip,temp,datetime
0,1,0.576628,17.865900,2017-01-01
1,1,0.166432,18.287195,2018-01-01
2,1,1.760069,19.013910,2019-01-01
3,1,0.000000,16.801691,2020-01-01
4,1,0.950271,17.495355,2021-01-01


## Calculate the components of R0: (1) Mosquito population survival per day

In [6]:
df['P'] = mos_survival(params['stype'], df['temp'].values)
df.head()

,forecast,precip,temp,datetime,P
0,1,0.576628,17.865900,2017-01-01,0.904054
1,1,0.166432,18.287195,2018-01-01,0.902434
2,1,1.760069,19.013910,2019-01-01,0.898305
3,1,0.000000,16.801691,2020-01-01,0.905616
4,1,0.950271,17.495355,2021-01-01,0.905009


## Calculate the components of R0: (2) Gonotrophic and Sporogonic cycle lengths

In [7]:
df['gono_len'] = gono_length(df['temp'].values, params['Dg'], params['Tg'])
df.head()

,forecast,precip,temp,datetime,P,gono_len
0,1,0.576628,17.865900,2017-01-01,0.904054,5.173293
1,1,0.166432,18.287195,2018-01-01,0.902434,4.983980
2,1,1.760069,19.013910,2019-01-01,0.898305,4.694860
3,1,0.000000,16.801691,2020-01-01,0.905616,5.742562
4,1,0.950271,17.495355,2021-01-01,0.905009,5.355321


In [8]:
df['sporo_len'] = sporo_length(df['temp'].values, params['Ds'], params['Ts'])
df.head()

,forecast,precip,temp,datetime,P,gono_len,sporo_len
0,1,0.576628,17.865900,2017-01-01,0.904054,5.173293,1000.000000
1,1,0.166432,18.287195,2018-01-01,0.902434,4.983980,386.497216
2,1,1.760069,19.013910,2019-01-01,0.898305,4.694860,109.477182
3,1,0.000000,16.801691,2020-01-01,0.905616,5.742562,1000.000000
4,1,0.950271,17.495355,2021-01-01,0.905009,5.355321,1000.000000


## Calculate the components of R0: (3) Mosquito population

In [9]:
df['datetime'].values.reshape((1,1,len(df)))

array([[['2017-01-01', '2018-01-01', '2019-01-01', ..., '2019-12-31',
         '2020-12-31', '2021-12-31']]], dtype=object)

In [ ]:
#forecast
p_array = df['P'].values.reshape((1,1,len(df)))
r_array = df['precip'].values.reshape((1,1,len(df)))
print(p_array.shape, r_array.shape)
m_array = calc_mosquito_pop(0.0, p_array, r_array, 1, params["rainfall_multiplier"], params["rainfall_offset"])
df['mosquitoes'] = m_array[0,0,:]
df.head(15)

## Bring it all together: Calculate R0

In [ ]:
#forecast
m_array = df['mosquitoes'].values.reshape(1,1,len(df))
p_array = df['P'].values.reshape((1,1,len(df)))
g_array = df['gono_len'].values.reshape((1,1,len(df)))
s_array = df['sporo_len'].values.reshape((1,1,len(df)))

In [ ]:
R0 = calculate_R0(m_array, p_array, g_array, s_array, params["r"], \
                             params["hia"], params["hbi"], params["b"], params["c"])
df['R0'] = R0[0,0,:]
df.head()